In [3]:
%config IPCompleter.greedy=True

In [45]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep, time
from random import randint
from IPython.core.display import clear_output
from warnings import warn
from urllib.parse import quote
import csv
import numpy
from math import ceil

In [31]:
series_names = []

with open('data/series_list_08-18-2020.csv','r') as series_data:
    csv_reader = csv.reader(series_data, delimiter=',')
    next(csv_reader, None)
    for row in csv_reader:
        series_names.append(row[2])

In [33]:
len(series_names)

829

In [77]:
series_slices = []

In [78]:
start = 0
end = 25
increment = 25
for i in range(ceil(len(series_names)/increment)):
    series_slices.append(slice(start, end))
    start = end
    end += increment

In [107]:
def search_imdb(data):
    start_time = time()
    requests = 0
    headers = {}
    
    names = []
    years = []
    imdb_ratings = []
    meta_scores = []
    votes = []
    
    for series in data:
    #     url = 'https://www.imdb.com/search/title/?title=Sunny+Bunnies&title_type=tv_series'
        params = quote(series)
        response = get(
            'https://www.imdb.com/search/title/?title={}&title_type=tv_series'.format(params),
            headers = headers)

        # pause the loop
        sleep(randint(8,15))

        # monitor the requests
        requests += 1
        elapsed_time = time() - start_time

        print('Request: {}; Timelapsed: {}; Progress: {:.2f}%'.format(requests, elapsed_time, (requests/len(data)) * 100))
        clear_output(wait=True)

        # throw waring for none 200 status
        if response.status_code !=200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            continue

        page_html = BeautifulSoup(response.text,'html.parser')
        mv_containers = page_html.find_all('div', class_='lister-item mode-advanced')[0:1]

     # iterate over movies on the page
        for container in mv_containers:
            if container.find('div', class_='ratings-metascore') is not None:
                metascore = container.find('div', class_='ratings-metascore').text
                meta_scores.append(metascore)

            # get name
            name = container.h3.a.text
            names.append(name)

            # get year
            year = container.h3.find('span', class_='lister-item-year').text
            years.append(year)

            # get imd rating
            try:
                imdb = float(container.strong.text)
            except:
                imdb = None
            imdb_ratings.append(imdb)
            
    data_set = pd.DataFrame({
        'series': names,
        'imdb': imdb_ratings,
        'years': years
    })
    return data_set

In [105]:
# as this is too large shall fetch in batches

In [99]:
### chunk 1
# controlling the crawl rate
# else will have ip blocked

data = series_names[series_slices[0]]
batch_1 = search_imdb(data)


Request: 25; Timelapsed: 310.59508419036865; Progress: 100.00%


In [100]:
batch_1

,series,imdb,years
0,A Test Series,6.1,(2016–2019)
1,Pucca,6.7,(2006–2008)
2,Gummibär & Friends: The Gummy Bear Show,3.5,(2016– )
3,Flying Rhino Junior High,6.7,(1998–2000)
4,Timothy Goes to School,6.4,(2000– )
5,Mother Goose Stories,8.0,(1990–1993)


In [108]:
# # chunk 2
data = series_names[series_slices[1]]
batch_2 = search_imdb(data)

Request: 25; Timelapsed: 297.8365559577942; Progress: 100.00%


In [109]:
batch_2

,series,imdb,years
0,The Ollie & Moon Show,8.2,(2017– )
1,Elvis and Benny,NaN,(2018– )
2,I Got a Rocket!,5.9,(2006–2007)
3,Sidekicks,7.3,(1986–1987)
4,Redwall,8.2,(1999– )
5,Jake and the Kid,7.6,(1995–1999)
6,The Koala Brothers,6.9,(2003– )
7,Mr. Young,6.1,(2011–2013)
